## LangChain — Structured Outputs with Ollama (DeepSeek R1 8B)

This notebook demonstrates how to return structured data from a model using LangChain's `with_structured_output` with `ChatOllama` and the `deepseek-r1:8b` model running locally via Ollama.

Reference: [LangChain — Structured Output](https://python.langchain.com/docs/how_to/structured_outputs/)

Assumptions:
- Ollama is running locally at `http://localhost:11434`
- The model `deepseek-r1:8b` is installed (run `ollama pull deepseek-r1:8b` if needed)


In [ ]:
# Optional: environment setup
python -V
pip install -qU langchain langchain-core langchain-ollama pydantic rich
# If you haven't yet: `ollama pull deepseek-r1:8b`


In [ ]:
from typing import List, Literal, Optional
from pydantic import BaseModel, Field

from langchain_ollama import ChatOllama

# Basic connectivity check to Ollama
llm = ChatOllama(model="deepseek-r1:8b", temperature=0)
resp = llm.invoke("Return the word 'ready'.")
print(resp.content)


In [ ]:
# Pydantic schema: Example 1 — Simple entity
class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline of the joke")

# Use JSON output mode to make the model emit pure JSON
json_llm = ChatOllama(model="deepseek-r1:8b", temperature=0, format="json")

# Bind structured output to the model
structured_llm = json_llm.with_structured_output(Joke)

joke: Joke = structured_llm.invoke(
    "Tell me a short, clean programming joke."
)

print(joke)
print("\nAs Python dict:", joke.model_dump())


In [ ]:
# Pydantic schema: Example 2 — Nested structure and enums
class Task(BaseModel):
    title: str
    priority: Literal["low", "medium", "high"]
    due_days: int = Field(ge=0, description="Days from today until due")

class TodoList(BaseModel):
    owner: str
    tasks: List[Task]
    notes: Optional[str] = None

structured_todos = json_llm.with_structured_output(TodoList)

todos: TodoList = structured_todos.invoke(
    "Create a to-do list for a developer preparing a LangChain demo. Return 3 tasks with varying priorities and realistic due days."
)

print(todos)
print("\nJSON:")
print(todos.model_dump_json(indent=2))


In [ ]:
# Example 3 — JSON Schema (dict) without Pydantic
# You can pass a raw JSON Schema and get back a Python dict.
event_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "description": "Event name"},
        "date": {"type": "string", "description": "ISO date"},
        "location": {"type": "string"},
        "attendees": {
            "type": "array",
            "items": {"type": "string"},
            "description": "List of attendee names"
        }
    },
    "required": ["name", "date", "location", "attendees"],
    "additionalProperties": False
}

structured_event = json_llm.with_structured_output(event_schema)

event = structured_event.invoke(
    "Create a small event for a local Python meetup with 3 attendees."
)

from pprint import pprint
pprint(event)


### Notes
- If `deepseek-r1:8b` returns reasoning text, using `format="json"` helps ensure pure JSON for structured output.
- For Pydantic models, the result is an instance with `.model_dump()` / `.model_dump_json()`.
- For raw JSON Schema, the result is a plain Python `dict`.
- Ensure Ollama is running: `ollama serve` and model pulled: `ollama pull deepseek-r1:8b`.
